In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import time
import logging
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

logger = logging.getLogger('azure.core.pipeline.policies.http_logging_policy').setLevel(logging.WARNING)

search_for_orig = "smart TV"

In [2]:
search_for_orig = search_for_orig.strip().lower()
amazonHeaders = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", 
                "Accept-Encoding":"gzip, deflate", 
                "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                "DNT":"1",
                "Connection":"close", 
                "Upgrade-Insecure-Requests":"1"}

flipkartHeaders = {"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0", 
                    "Accept-Encoding":"gzip, deflate", 
                    "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                    "DNT":"1",
                    "Connection":"close", 
                    "Upgrade-Insecure-Requests":"1"
                    }

num_prod_search = 160
displayBox = None
refresh = False

In [3]:
platform = "Flipkart"
search_for = search_for_orig.replace(' ', '%20')
base_link = ("https://www.flipkart.com/search?q=" + search_for 
                + 
            "&sort=popularity&p%5B%5D=facets.fulfilled_by%255B%255D%3DPlus%2B%2528FAssured%2529" +
            "&p%5B%5D=facets.rating%255B%255D%3D4%25E2%2598%2585%2B%2526%2Babove" +
            "&p%5B%5D=facets.availability%255B%255D%3DExclude%2BOut%2Bof%2BStock"
            )

link = base_link+"&page="+str(1)

response = session.get(link)
html_text = response.text
soup = BeautifulSoup(html_text, "html.parser")

allRows = soup.find_all('div', class_='_75nlfW')

In [3]:
FlipAllRows = '_75nlfW'

FlipWid100Desc = 'KzDlHZ'
FlipWid100Prodlnk = 'CGtC98'
FlipWid100Pricebox = 'Nx9bqj _4b5DiR'
FlipWid100Ratebox = 'XQDdHH'
FlipWid100Ratedata = "Wphh3N"
FlipWid100Imgurl = "DByuf4"

FlipWid25Cat1ProdsInRow = 'slAVV4'
FlipWid25Cat1Header = 'wjcEIp'
FlipWid25Cat1Pricebox = 'Nx9bqj'
FlipWid25Cat1Ratebox = 'XQDdHH'
FlipWid25Cat1Raters = 'Wphh3N'
FlipWid25Cat1Imgurl = 'DByuf4'

FlipWid25Cat2ProdsInRow = '_1sdMkc'
FlipWid25Cat2Header = 'WKTcLC'
FlipWid25Cat2Pricebox = 'Nx9bqj'
FlipWid25Cat2Ratebox = 'XQDdHH'
FlipWid25Cat2Raters = 'Wphh3N'
FlipWid25Cat2Imgurl = '_53J4C'

df_list = []

platform = "Flipkart"
search_for = search_for_orig.replace(' ', '%20')
base_link = ("https://www.flipkart.com/search?q=" + search_for 
                + 
            "&sort=popularity&p%5B%5D=facets.fulfilled_by%255B%255D%3DPlus%2B%2528FAssured%2529" +
            "&p%5B%5D=facets.rating%255B%255D%3D4%25E2%2598%2585%2B%2526%2Babove" +
            "&p%5B%5D=facets.availability%255B%255D%3DExclude%2BOut%2Bof%2BStock"
            )

style = ''
progress = 0.0
page2 = 0
num_prod2 = 0
while progress < 100:
    page2 = page2 + 1
    if page2 == 10:
        break   
    link = base_link+"&page="+str(page2)
    status = 0
    req_num = 0
    while (status!=200) & (req_num < 30):
        req_num = req_num+1
        response = session.get(link, headers=flipkartHeaders)
        status = response.status_code
        print(f"attempt number {req_num}, for page number {page2}, received status {status} in last attempt")
        # time.sleep(0.01*req_num)
    html_text = response.text
    soup = BeautifulSoup(html_text, "html.parser")

    # status_write.write(soup[:50])
    allRows = soup.find_all('div', class_=FlipAllRows)
    print(f"checking {len(allRows)} rows for page number {page2}")
    for row in allRows:
        if style == '':
            style = row.find('div')['style']
        
        if style == 'width:100%':
            descrs = row.find('div', class_=FlipWid100Desc).text
            prodLink = 'https://www.flipkart.com' + (row.find('a', class_=FlipWid100Prodlnk)['href'].split('?')[0])
            pricebox = row.find('div', class_=FlipWid100Pricebox)
            if pricebox is None:
                print("continuing at pricebox")
                continue
            price = int(pricebox.text[1:].replace(',', ''))
            ratebox = row.find('div', class_=FlipWid100Ratebox)
            if ratebox is None:
                print("continuing at ratebox")
                continue
            rating = float(ratebox.text)
            rateData = row.find('span', class_=FlipWid100Ratedata).text
            raters = int(rateData.split()[0].replace(',',''))
            if raters < 30:
                print("continuing at raters")
                continue
            reviewers = int(rateData.split()[3].replace(',',''))
            img_url = row.find('img', class_=FlipWid100Imgurl)['src']
            df_list.append([platform, descrs, prodLink, price, rating, raters, reviewers, img_url])
            num_prod2 = num_prod2 + 1
            print(f"Flipkart : searched for {num_prod2} products in {page2} pages")
            progress = progress + (100/(num_prod_search))
            continue
            
        elif style == 'width:25%':
            prods = row.find_all('div', class_=FlipWid25Cat1ProdsInRow)
            if len(prods) > 0:
                for prod in prods:
                    header = prod.find('a', class_=FlipWid25Cat1Header)
                    descrs = header['title']
                    prodLink = 'https://www.flipkart.com' + (header['href'].split('?')[0]) + '?marketplace=FLIPKART'
                    pricebox = prod.find('div', class_=FlipWid25Cat1Pricebox)
                    if pricebox is None:
                        print("continuing at pricebox")
                        continue
                    price = int(pricebox.text[1:].replace(',', ''))
                    ratebox = prod.find('div', class_=FlipWid25Cat1Ratebox)
                    if ratebox is None:
                        print("continuing at ratebox")
                        continue
                    rating = float(ratebox.text)
                    raters = int(prod.find('span', class_=FlipWid25Cat1Raters).text[1:-1].replace(',',''))
                    if raters < 30:
                        print("continuing at raters")
                        continue
                    img_url = prod.find('img', class_=FlipWid25Cat1Imgurl)['src']
                    df_list.append([platform, descrs, prodLink, price, rating, raters, np.nan, img_url])
                    num_prod2 = num_prod2 + 1
                    print(f"Flipkart : searched for {num_prod2} products in {page2} pages")
                    progress = progress + (100/(num_prod_search))
            else:
                prods = row.find_all('div', class_=FlipWid25Cat2ProdsInRow)
                for prod in prods:
                    header = prod.find('a', class_=FlipWid25Cat2Header)
                    descrs = header['title']
                    prodLink = 'https://www.flipkart.com' + (header['href'].split('?')[0]) + '?marketplace=FLIPKART'
                    price = int(prod.find('div', class_=FlipWid25Cat2Pricebox).text[1:].replace(',', ''))
                    prod_text = session.get(prodLink, headers=flipkartHeaders).text
                    prod_soup = BeautifulSoup(prod_text, "html.parser")
                    ratebox = prod_soup.find('div', class_=FlipWid25Cat2Ratebox)
                    if ratebox is None:
                        print("continuing at ratebox")
                        continue
                    rating = float(ratebox.text)
                    rateData = prod_soup.find('span', class_=FlipWid25Cat1Raters)
                    if (rateData is None) or (len(rateData.text.split()) < 4):
                        print("continuing at ratedata")
                        continue
                    raters = int(rateData.text.split()[0].replace(',',''))
                    if raters < 30:
                        print("continuing at raters")
                        continue
                    reviewers = int(rateData.text.split()[3].replace(',',''))
                    img_url = prod.find('img', class_=FlipWid25Cat2Imgurl)['src']
                    df_list.append([platform, descrs, prodLink, price, rating, raters, reviewers, img_url])
                    num_prod2 = num_prod2 + 1
                    print(f"Flipkart : searched for {num_prod2} products in {page2} pages")
                    progress = progress + (100/(0.3*num_prod_search))
            
            
df = pd.DataFrame(df_list, columns=['platform', 'Desc', 'Link', 'Price', 'Rating', 'Raters', 'Reviewers', 'img_url'])
df = df.drop_duplicates(subset=['Price', 'Rating', 'Raters'])
df = df[(((df["Price"] > df["Price"].quantile(0.025)) & 
            (df["Price"] < df["Price"].quantile(0.985)) &
            (df["platform"]=="Amazon")) |
            (df["platform"]=="Flipkart")
            )]
df['Scaled Rating'] = df['Rating']*(1 - np.power(1.25, -1*np.sqrt(df['Raters'])))
df['VFM'] = (df['Scaled Rating']/(df['Scaled Rating'].median()))*(np.sqrt(df['Price'].median()))/np.sqrt(df['Price'])
df['composite'] = ((df['Scaled Rating']/(df['Scaled Rating'].median())) * (np.sqrt(df['VFM'])/(np.sqrt(df['VFM'].median()))))

attempt number 1, for page number 1, received status 200 in last attempt
checking 24 rows for page number 1
Flipkart : searched for 1 products in 1 pages
Flipkart : searched for 2 products in 1 pages
Flipkart : searched for 3 products in 1 pages
Flipkart : searched for 4 products in 1 pages
Flipkart : searched for 5 products in 1 pages
Flipkart : searched for 6 products in 1 pages
Flipkart : searched for 7 products in 1 pages
Flipkart : searched for 8 products in 1 pages
Flipkart : searched for 9 products in 1 pages
Flipkart : searched for 10 products in 1 pages
Flipkart : searched for 11 products in 1 pages
Flipkart : searched for 12 products in 1 pages
Flipkart : searched for 13 products in 1 pages
Flipkart : searched for 14 products in 1 pages
Flipkart : searched for 15 products in 1 pages
Flipkart : searched for 16 products in 1 pages
Flipkart : searched for 17 products in 1 pages
Flipkart : searched for 18 products in 1 pages
Flipkart : searched for 19 products in 1 pages
Flipkart

In [4]:
df_list

[['Flipkart',
  'Mi A Series 80 cm (32 inch) HD Ready LED Smart Google TV 2024 Edition',
  'https://www.flipkart.com/mi-series-80-cm-32-inch-hd-ready-led-smart-google-tv-2024/p/itm2b53ae95cfc5b',
  12499,
  4.3,
  47654,
  3668,
  'https://rukminim2.flixcart.com/image/312/312/xif0q/television/b/i/b/l32ma-ain-mi-original-imah2pgfzpg3zuys.jpeg?q=70'],
 ['Flipkart',
  'LG 80 cm (32 inch) HD Ready LED Smart WebOS TV with a5 AI Processor Gen5, Refresh Rate: 60 hertz, Magi...',
  'https://www.flipkart.com/lg-80-cm-32-inch-hd-ready-led-smart-webos-tv-a5-ai-processor-gen5-refresh-rate-60-hertz-magic-remote-capability/p/itmf3b681525947c',
  15988,
  4.3,
  13635,
  1038,
  'https://rukminim2.flixcart.com/image/312/312/xif0q/television/3/x/9/-original-imah2sstwrujqyp4.jpeg?q=70'],
 ['Flipkart',
  'Thomson Alpha 60 cm (24 Inch) HD Ready LED Smart Linux TV with 20 W Sound Output',
  'https://www.flipkart.com/thomson-alpha-60-cm-24-inch-hd-ready-led-smart-linux-tv-20-w-sound-output/p/itm8e1f318f88c

In [6]:
link

'https://www.flipkart.com/search?q=smart%20tv&sort=popularity&p%5B%5D=facets.fulfilled_by%255B%255D%3DPlus%2B%2528FAssured%2529&p%5B%5D=facets.rating%255B%255D%3D4%25E2%2598%2585%2B%2526%2Babove&p%5B%5D=facets.availability%255B%255D%3DExclude%2BOut%2Bof%2BStock&page=2'

In [15]:
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)
flipkartHeaders = {"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0", 
                    "Accept-Encoding":"gzip, deflate", 
                    "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                    "DNT":"1",
                    "Connection":"close", 
                    "Upgrade-Insecure-Requests":"1"
                    }

In [5]:
link = base_link+"&page="+str(3)

In [6]:
response = session.get(link, headers=flipkartHeaders)

In [7]:
response.status_code

200

In [8]:
html_text = response.text
soup = BeautifulSoup(html_text, "html.parser")

# status_write.write(soup[:50])
allRows = soup.find_all('div', class_=FlipAllRows)

In [11]:
descrs = allRows[0].find('div', class_=FlipWid100Desc).text

In [12]:
descrs

'KODAK 108 cm (43 inch) Full HD LED Smart Linux TV 2024 Edition with 30W Sound Output & Bezel-Less Desi...'

In [7]:
allRows = soup.find_all('div', class_='_75nlfW')
prods = allRows[0]
prods = row.find_all('div', class_='_4ddWXP')


KeyError: 0

In [ ]:
FlipWid25Header = 'wjcEIp'

# FlipWid100Desc = 'KzDlHZ'
# FlipWid100Prodlnk = 'CGtC98'
# FlipWid100Pricebox = 'Nx9bqj _4b5DiR'
# FlipWid100Ratebox = 'XQDdHH'
# FlipWid100Ratedata = "Wphh3N"
# FlipWid100Imgurl = "DByuf4"

In [6]:
prod.find('a', class_=FlipWid25Header)

NameError: name 'prod' is not defined

In [88]:
allRows

[<div class="_75nlfW LYgYA3"><div data-id="WATF3DEDG52JR2TC" style="width:25%"><div class="_1sdMkc LFEi7Z"><a class="rPDeLR" href="/sonata-volt-analog-watch-men/p/itmf3zhmqygtfkjz?pid=WATF3DEDG52JR2TC&amp;lid=LSTWATF3DEDG52JR2TCTR8UWD&amp;marketplace=FLIPKART&amp;q=watch&amp;store=r18%2Ff13&amp;spotlightTagId=BestsellerId_r18%2Ff13&amp;srno=s_1_1&amp;otracker=search&amp;fm=organic&amp;iid=af33f7f9-0c83-4fea-baa2-2b4f8848639f.WATF3DEDG52JR2TC.SEARCH&amp;ppt=None&amp;ppn=None&amp;ssid=vrvge8wvfk0000001720986137772&amp;qH=d2974c96dc96b3f3" rel="noopener noreferrer" target="_blank"><div><div><div class="wvIX4U" style="padding-top:120.00%"><div class="gqcSqV YGE0gZ" style="padding-top:120.00%"><img alt="" class="_53J4C-" src="https://rukminim2.flixcart.com/image/612/612/xif0q/watch/i/q/e/-original-imagnre8pbpnfhgz.jpeg?q=70"/></div></div></div></div><div></div><div class="oUss6M D05kjV gREhVj"><div class="+7E521"><svg class="N1bADF" height="28" viewbox="0 0 20 16" width="28" xmlns="http://w

In [89]:
row = allRows[0]

In [81]:
prod = row.find_all('div', class_='_1sdMkc')[0]

In [83]:
prod

<div class="_1sdMkc LFEi7Z"><a class="rPDeLR" href="/r-enterprises-alloy-rhodium-silver-jewellery-set/p/itm521ed40577b18?pid=JWSG4DYPAGFFUCGF&amp;lid=LSTJWSG4DYPAGFFUCGFA2GV77&amp;marketplace=FLIPKART&amp;q=bracelet&amp;store=mcr&amp;spotlightTagId=BestsellerId_mcr&amp;srno=s_1_1&amp;otracker=search&amp;fm=organic&amp;iid=593cf10d-fa91-4ca0-84d2-18ab6e1c99ae.JWSG4DYPAGFFUCGF.SEARCH&amp;ppt=None&amp;ppn=None&amp;ssid=l888q2qzyo0000001720984061276&amp;qH=992df610cee45108" rel="noopener noreferrer" target="_blank"><div><div><div class="wvIX4U" style="padding-top:120.00%"><div class="gqcSqV YGE0gZ" style="padding-top:120.00%"><img alt="" class="_53J4C-" src="https://rukminim2.flixcart.com/image/612/612/xif0q/shopsy-jewellery-set/p/l/q/sps-barito-earring-necklace-set-alasca-original-imahynyxayverfez.jpeg?q=70"/></div></div></div></div><div class="jt22zr" style="bottom:4px"></div><div></div><div class="oUss6M D05kjV gREhVj"><div class="+7E521"><svg class="N1bADF" height="28" viewbox="0 0 20 

In [31]:
header = prod.find('a', class_='wjcEIp')
header

<a class="wjcEIp" href="/nova-nht-1052-usb-trimmer-90-min-runtime-40-length-settings/p/itm08a4a983d35d1?pid=TMRFZ9NQWGHEKQBS&amp;lid=LSTTMRFZ9NQWGHEKQBSBZBZHT&amp;marketplace=FLIPKART&amp;q=trimmer&amp;store=zlw%2F79s%2Fby3&amp;spotlightTagId=BestsellerId_zlw%2F79s%2Fby3&amp;srno=s_1_1&amp;otracker=search&amp;fm=organic&amp;iid=ab49766b-706d-46ca-b622-6df3b72a437a.TMRFZ9NQWGHEKQBS.SEARCH&amp;ppt=None&amp;ppn=None&amp;ssid=rauj8cspvk0000001720977412878&amp;qH=705a17deac7a99db" rel="noopener noreferrer" target="_blank" title="NOVA NHT 1052 USB Trimmer 90 min  Runtime 40 Length Settings">NOVA NHT 1052 USB Trimmer 90 min  Runtime 40 Length Set...</a>

In [32]:
header["title"]

'NOVA NHT 1052 USB Trimmer 90 min  Runtime 40 Length Settings'

In [94]:
row = allRows[0]
prod = row.find_all('div', class_='_1sdMkc')[0]
header = prod.find('a', class_='WKTcLC')
descrs = header['title']
prodLink = 'https://www.flipkart.com' + (header['href'].split('?')[0]) + '?marketplace=FLIPKART'
price = int(prod.find('div', class_='Nx9bqj').text[1:].replace(',', ''))
# sleep(1.5)
prod_text = session.get(prodLink).text
prod_soup = BeautifulSoup(prod_text, "html.parser")
# ratebox = prod_soup.find('div', class_='_3LWZlK _3uSWvT')
ratebox = prod_soup.find('div', class_='XQDdHH')
if ratebox is None:
    print("continuing at ratebox")
rating = float(ratebox.text)
rateData = prod_soup.find('span', class_="Wphh3N")
if (rateData is None) or (len(rateData.text.split()) < 4):
    print("continuing at ratedata")
raters = int(rateData.text.split()[0].replace(',',''))
if raters < 30:
    print("continuing at raters")
reviewers = int(rateData.text.split()[3].replace(',',''))
img_url = prod.find('img', class_="_53J4C-")['src']

In [95]:
img_url

'https://rukminim2.flixcart.com/image/612/612/xif0q/watch/i/q/e/-original-imagnre8pbpnfhgz.jpeg?q=70'

In [93]:
session.get(prodLink+'?marketplace=FLIPKART').text

'<!doctype html><html lang="en"><head><link href="https://rukminim2.flixcart.com" rel="preconnect"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/atlas.chunk.079771.css"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.d32cf2.css"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.89b63e.css"/><meta http-equiv="Content-type" content="text/html; charset=utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=Edge"/><meta property="fb:page_id" content="102988293558"/><meta property="fb:admins" content="658873552,624500995,100000233612389"/><link rel="shortcut icon" href="https:///www/promos/new/20150528-140547-favicon-retina.ico"/><link type="application/opensearchdescription+xml" rel="search" href="/osdd.xml?v=2"/><meta property="og:type" content="website"/><meta name="og_site_name" property="og:site_name" content="Flipka

In [77]:
price

199

In [64]:
prod_text = session.get(prodLink).text

In [66]:
prod_soup = BeautifulSoup(prod_text, "html.parser")

In [71]:
ratebox = prod_soup.find('div', class_='XQDdHH')

In [73]:
ratebox.text

'4'

In [55]:
prodLink

'https://www.flipkart.com/r-enterprises-alloy-rhodium-silver-jewellery-set/p/itm521ed40577b18'

In [53]:
price

199

In [7]:
with open("temp.txt","r") as f:
    html = f.read()

In [8]:
soup = BeautifulSoup(html, "html.parser")

In [15]:
print(soup.prettify())

https://www.flipkart.com/search?q=smart%20tv&amp;sort=popularity&amp;p%5B%5D=facets.fulfilled_by%255B%255D%3DPlus%2B%2528FAssured%2529&amp;p%5B%5D=facets.rating%255B%255D%3D4%25E2%2598%2585%2B%2526%2Babove&amp;p%5B%5D=facets.availability%255B%255D%3DExclude%2BOut%2Bof%2BStock&amp;page=1
<!DOCTYPE html>
<html lang="en">
 <head>
  <link href="https://rukminim2.flixcart.com" rel="preconnect"/>
  <link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/atlas.chunk.079771.css" rel="stylesheet"/>
  <link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.d32cf2.css" rel="stylesheet"/>
  <link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.89b63e.css" rel="stylesheet"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="102988293558" property="fb:page_id"/>
  <meta content="658873552,624500995,1000002336123

In [6]:
soup.find_all('div', class_='_13oc-S')

[]

In [45]:
row.find('img', class_="DByuf4")["src"]

'https://rukminim2.flixcart.com/image/312/312/xif0q/television/b/s/o/-original-imagt86kchfhpwgz.jpeg?q=70'

In [46]:
FlipWid100Desc = 'KzDlHZ'
FlipWid100Prodlnk = 'CGtC98'
FlipWid100Pricebox = 'Nx9bqj _4b5DiR'
FlipWid100Ratebox = 'XQDdHH'
FlipWid100Ratedata = "Wphh3N"
FlipWid100Imgurl = "DByuf4"

In [47]:
descrs = row.find('div', class_=FlipWid100Desc).text
prodLink = 'https://www.flipkart.com' + (row.find('a', class_=FlipWid100Prodlnk)['href'].split('?')[0])
pricebox = row.find('div', class_=FlipWid100Pricebox)
if pricebox is None:
    print("continuing at pricebox")
    # continue
price = int(pricebox.text[1:].replace(',', ''))
ratebox = row.find('div', class_=FlipWid100Ratebox)
if ratebox is None:
    print("continuing at ratebox")
    # continue
rating = float(ratebox.text)
rateData = row.find('span', class_=FlipWid100Ratedata).text
raters = int(rateData.split()[0].replace(',',''))
if raters < 30:
    print("continuing at raters")
    # continue
reviewers = int(rateData.split()[3].replace(',',''))
img_url = row.find('img', class_=FlipWid100Imgurl)["src"]

In [2]:
import streamlit as st
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import time
from azure.storage.blob import BlobServiceClient
import logging


logger = logging.getLogger('azure.core.pipeline.policies.http_logging_policy').setLevel(logging.WARNING)

connection_string = st.secrets["database_connection"]["connection_string"]
container_name = st.secrets["database_connection"]["container_name"]
masterSearchFileName = st.secrets["database_connection"]["masterSearchFileName"]
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container=container_name)

downlodData = container_client.download_blob(masterSearchFileName).readall()
keydict = pd.read_json(BytesIO(downlodData), dtype={"Search":str, "Time":np.float32, "Key":str})

search_for_orig = "5g smartphone"
# search_for_orig = st.text_input(label='Search for:', value='')
search_for_orig = search_for_orig.strip().lower()

num_prod_search = 160
displayBox = None
refresh = False

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", 
           "Accept-Encoding":"gzip, deflate", 
           "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
           "DNT":"1",
           "Connection":"close", 
           "Upgrade-Insecure-Requests":"1"}

In [3]:
link="https://www.flipkart.com/search?q=trimmer&sort=popularity&p%5B%5D=facets.fulfilled_by%255B%255D%3DPlus%2B%2528FAssured%2529&p%5B%5D=facets.rating%255B%255D%3D4%25E2%2598%2585%2B%2526%2Babove&p%5B%5D=facets.availability%255B%255D%3DExclude%2BOut%2Bof%2BStock&page=3"

In [12]:
response = requests.get(link, headers=headers)

In [13]:
response.status_code

200

In [14]:
print(response.text)

<!doctype html><html lang="en"><head><link href="https://rukminim2.flixcart.com" rel="preconnect"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.09b0e9.css"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.e82689.css"/><meta http-equiv="Content-type" content="text/html; charset=utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=Edge"/><meta property="fb:page_id" content="102988293558"/><meta property="fb:admins" content="658873552,624500995,100000233612389"/><link rel="shortcut icon" href="https://static-assets-web.flixcart.com/www/promos/new/20150528-140547-favicon-retina.ico"/><link type="application/opensearchdescription+xml" rel="search" href="/osdd.xml?v=2"/><meta property="og:type" content="website"/><meta name="og_site_name" property="og:site_name" content="Flipkart.com"/><link rel="apple-touch-icon" sizes="57x57" href="/apple-touch-icon-57x57.png"/><li

In [15]:
platform = "Flipkart"
search_for = search_for_orig.replace(' ', '%20')
base_link = ("https://www.flipkart.com/search?q=" + search_for + 
            "&sort=popularity&p%5B%5D=facets.fulfilled_by%255B%255D%3DPlus%2B%2528FAssured%2529" +
            "&p%5B%5D=facets.rating%255B%255D%3D4%25E2%2598%2585%2B%2526%2Babove" +
            "&p%5B%5D=facets.availability%255B%255D%3DExclude%2BOut%2Bof%2BStock")
df_list = []
style = ''
progress = 0.0
page2 = 0
num_prod2 = 0
flp_write = st.empty()
flp_write.write("Flipkart : ")
my_bar2 = st.progress(int(progress))
while progress < 100:
    page2 = page2 + 1
    if page2 == 10:
        my_bar2.progress(100)
        break   
    link = base_link+"&page="+str(page2)
    html_text = requests.get(link, headers=headers).text
    soup = BeautifulSoup(html_text, "html.parser")

    for row in soup.find_all('div', class_='_13oc-S'):
        if style == '':
            style = row.find('div')['style']
        
        if style == 'width:100%':
            descrs = row.find('div', class_='_4rR01T').text
            prodLink = 'https://www.flipkart.com' + (row.find('a', class_='_1fQZEK')['href'].split('?')[0])
            pricebox = row.find('div', class_='_30jeq3 _1_WHN1')
            if pricebox is None:
                continue
            price = int(pricebox.text[1:].replace(',', ''))
            ratebox = row.find('div', class_='_3LWZlK')
            if ratebox is None:
                continue
            rating = float(ratebox.text)
            rateData = row.find('span', class_="_2_R_DZ").text
            raters = int(rateData.split()[0].replace(',',''))
            if raters < 30:
                continue
            reviewers = int(rateData.split()[3].replace(',',''))
            img_url = row.find('img', class_="_396cs4")['src']
            df_list.append([platform, descrs, prodLink, price, rating, raters, reviewers, img_url])
            num_prod2 = num_prod2 + 1
            flp_write.write(f"Flipkart : searched for {num_prod2} products in {page2} pages")
            progress = progress + (100/(num_prod_search))
            my_bar2.progress(int(progress) if int(progress)<100 else 100)
            continue
            
        elif style == 'width:25%':
            prods = row.find_all('div', class_='_4ddWXP')
            if len(prods) > 0:
                for prod in prods:
                    header = prod.find('a', class_='s1Q9rs')
                    descrs = header['title']
                    prodLink = 'https://www.flipkart.com' + (header['href'].split('?')[0])
                    pricebox = prod.find('div', class_='_30jeq3')
                    if pricebox is None:
                        continue
                    price = int(pricebox.text[1:].replace(',', ''))
                    ratebox = prod.find('div', class_="_3LWZlK")
                    if ratebox is None:
                        continue
                    rating = float(ratebox.text)
                    raters = int(prod.find('span', class_="_2_R_DZ").text[1:-1].replace(',',''))
                    if raters < 30:
                        continue
                    img_url = prod.find('img', class_="_396cs4")['src']
                    df_list.append([platform, descrs, prodLink, price, rating, raters, np.nan, img_url])
                    num_prod2 = num_prod2 + 1
                    flp_write.write(f"Flipkart : searched for {num_prod2} products in {page2} pages")
                    progress = progress + (100/(num_prod_search))
                    my_bar2.progress(int(progress) if int(progress)<100 else 100)
            else:
                prods = row.find_all('div', class_='_1xHGtK _373qXS')
                for prod in prods:
                    header = prod.find('a', class_='IRpwTa')
                    descrs = header['title']
                    prodLink = 'https://www.flipkart.com' + (header['href'].split('?')[0])
                    price = int(prod.find('div', class_='_30jeq3').text[1:].replace(',', ''))
                    # sleep(1.5)
                    prod_text = requests.get(prodLink).text
                    prod_soup = BeautifulSoup(prod_text, "html.parser")
                    # ratebox = prod_soup.find('div', class_='_3LWZlK _3uSWvT')
                    ratebox = prod_soup.find('div', class_='_3LWZlK')
                    if ratebox is None:
                        continue
                    rating = float(ratebox.text)
                    rateData = prod_soup.find('span', class_="_2_R_DZ")
                    if (rateData is None) or (len(rateData.text.split()) < 4):
                        continue
                    raters = int(rateData.text.split()[0].replace(',',''))
                    if raters < 30:
                        continue
                    reviewers = int(rateData.text.split()[3].replace(',',''))
                    img_url = prod.find('img', class_="_2r_T1I")['src']
                    df_list.append([platform, descrs, prodLink, price, rating, raters, reviewers, img_url])
                    num_prod2 = num_prod2 + 1
                    flp_write.write(f"Flipkart : searched for {num_prod2} products in {page2} pages")
                    progress = progress + (100/(0.3*num_prod_search))
                    my_bar2.progress(int(progress) if int(progress)<100 else 100)

In [13]:
prodLink

'https://www.flipkart.com/redmi-12-5g-pastel-blue-128-gb/p/itm78076aae16588'

In [14]:
row.find('div', class_='_30jeq3 _1_WHN1')

In [9]:
link

'https://www.flipkart.com/search?q=5g%20smartphone&sort=popularity&p%5B%5D=facets.fulfilled_by%255B%255D%3DPlus%2B%2528FAssured%2529&p%5B%5D=facets.rating%255B%255D%3D4%25E2%2598%2585%2B%2526%2Babove&p%5B%5D=facets.availability%255B%255D%3DExclude%2BOut%2Bof%2BStock&page=2'

In [17]:
len(df_list)

168

In [3]:
import streamlit as st
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import time
from azure.storage.blob import BlobServiceClient
import logging


search_for_orig = "neck support travelling pillow"

num_prod_search = 160
displayBox = None
refresh = False
        

In [4]:
platform = "Flipkart"
search_for = search_for_orig.replace(' ', '%20')
base_link = ("https://www.flipkart.com/search?q=" + search_for + 
            "&sort=popularity&p%5B%5D=facets.fulfilled_by%255B%255D%3DPlus%2B%2528FAssured%2529" +
            "&p%5B%5D=facets.rating%255B%255D%3D4%25E2%2598%2585%2B%2526%2Babove" +
            "&p%5B%5D=facets.availability%255B%255D%3DExclude%2BOut%2Bof%2BStock")

In [6]:
style = ''
progress = 0.0
page2 = 0
num_prod2 = 0
page2 = page2 + 1
link = base_link+"&page="+str(page2)
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/605.1.15", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"} 
html_text = requests.get(link, headers=headers).text
soup = BeautifulSoup(html_text, "html.parser")

In [7]:
df_list = []

In [27]:
for row in soup.find_all('div', class_='_13oc-S'):
    if style == '':
        style = row.find('div')['style']
    
    if style == 'width:100%':
        descrs = row.find('div', class_='_4rR01T').text
        prodLink = 'https://www.flipkart.com' + (row.find('a', class_='_1fQZEK')['href'].split('?')[0])
        price = int(row.find('div', class_='_30jeq3 _1_WHN1').text[1:].replace(',', ''))
        ratebox = row.find('div', class_='_3LWZlK')
        if ratebox == None:
            continue
        rating = float(ratebox.text)
        rateData = row.find('span', class_="_2_R_DZ").text
        raters = int(rateData.split()[0].replace(',',''))
        if raters < 30:
            continue
        reviewers = int(rateData.split()[3].replace(',',''))
        img_url = row.find('img', class_="_396cs4")['src']
        df_list.append([platform, descrs, prodLink, price, rating, raters, reviewers, img_url])
        num_prod2 = num_prod2 + 1
        progress = progress + (100/(num_prod_search))
        continue
        
    elif style == 'width:25%':
        prods = row.find_all('div', class_='_4ddWXP')
        if len(prods) > 0:
            for prod in prods:
                header = prod.find('a', class_='s1Q9rs')
                descrs = header['title']
                prodLink = 'https://www.flipkart.com' + (header['href'].split('?')[0])
                pricebox = prod.find('div', class_='_30jeq3')
                if pricebox == None:
                    continue
                price = int(pricebox.text[1:].replace(',', ''))
                ratebox = prod.find('div', class_="_3LWZlK")
                if ratebox == None:
                    continue
                rating = float(ratebox.text)
                raters = int(prod.find('span', class_="_2_R_DZ").text[1:-1].replace(',',''))
                if raters < 30:
                    continue
                img_url = prod.find('img', class_="_396cs4")['src']
                df_list.append([platform, descrs, prodLink, price, rating, raters, np.nan, img_url])
                num_prod2 = num_prod2 + 1
                progress = progress + (100/(num_prod_search))
        else:
            prods = row.find_all('div', class_='_1xHGtK _373qXS')
            print("I m in 2")
            for prod in prods:
                header = prod.find('a', class_='IRpwTa')
                descrs = header['title']
                prodLink = 'https://www.flipkart.com' + (header['href'].split('?')[0])
                price = int(prod.find('div', class_='_30jeq3').text[1:].replace(',', ''))
                # sleep(1.5)
                prod_text = requests.get(prodLink).text
                prod_soup = BeautifulSoup(prod_text, "html.parser")
                # ratebox = prod_soup.find('div', class_='_3LWZlK _3uSWvT')
                ratebox = prod_soup.find('div', class_='_3LWZlK')
                if ratebox == None:
                    continue
                rating = float(ratebox.text)
                rateData = prod_soup.find('span', class_="_2_R_DZ")
                if (rateData == None) or (len(rateData.text.split()) < 4):
                    continue
                raters = int(rateData.text.split()[0].replace(',',''))
                if raters < 30:
                    continue
                reviewers = int(rateData.text.split()[3].replace(',',''))
                img_url = prod.find('img', class_="_2r_T1I")['src']
                df_list.append([platform, descrs, prodLink, price, rating, raters, reviewers, img_url])
                num_prod2 = num_prod2 + 1
                progress = progress + (100/(0.3*num_prod_search))            
            
df = pd.DataFrame(df_list, columns=['platform', 'Desc', 'Link', 'Price', 'Rating', 'Raters', 'Reviewers', 'img_url'])

I m in 2
I m in 2
I m in 2
I m in 2
I m in 2
I m in 2
I m in 2
I m in 2
I m in 2
I m in 2


In [12]:
prodLink

'https://www.flipkart.com/redmi-12-5g-pastel-blue-128-gb/p/itm78076aae16588'

In [28]:
df

,platform,Desc,Link,Price,Rating,Raters,Reviewers,img_url
0,Flipkart,Travel Neck Pillow,https://www.flipkart.com/ever-n-green-travel-n...,199,4.1,4291,297,https://rukminim2.flixcart.com/image/612/612/k...
1,Flipkart,Neck Pillow Travel Head Pillows Adjustable Pro...,https://www.flipkart.com/syga-neck-pillow-trav...,449,4.1,275,12,https://rukminim2.flixcart.com/image/612/612/x...
2,Flipkart,Neck Pillow Travel Head Pillows Adjustable Pro...,https://www.flipkart.com/syga-neck-pillow-trav...,499,4.1,912,78,https://rukminim2.flixcart.com/image/612/612/k...
3,Flipkart,Travel Neck Pillow,https://www.flipkart.com/ever-n-green-travel-n...,199,4.1,4291,297,https://rukminim2.flixcart.com/image/612/612/k...
4,Flipkart,Neck Pillow Travel Head Pillows Adjustable Pro...,https://www.flipkart.com/syga-neck-pillow-trav...,449,4.1,275,12,https://rukminim2.flixcart.com/image/612/612/x...
5,Flipkart,Neck Pillow Travel Head Pillows Adjustable Pro...,https://www.flipkart.com/syga-neck-pillow-trav...,499,4.1,912,78,https://rukminim2.flixcart.com/image/612/612/k...
6,Flipkart,Travel Neck Pillow,https://www.flipkart.com/ever-n-green-travel-n...,199,4.1,4291,297,https://rukminim2.flixcart.com/image/612/612/k...
7,Flipkart,Neck Pillow Travel Head Pillows Adjustable Pro...,https://www.flipkart.com/syga-neck-pillow-trav...,449,4.1,275,12,https://rukminim2.flixcart.com/image/612/612/x...
8,Flipkart,Neck Pillow Travel Head Pillows Adjustable Pro...,https://www.flipkart.com/syga-neck-pillow-trav...,499,4.1,912,78,https://rukminim2.flixcart.com/image/612/612/k...
9,Flipkart,Micro Fibre U-Shaped Soft Travelling Pillow Se...,https://www.flipkart.com/everyonic-micro-fibre...,294,4.2,340,24,https://rukminim2.flixcart.com/image/612/612/k...
